##Linear Regression Exercise

In this exercise, we will be predicting the median housing price in an area given certain attributes that describe area. We will be working with california housing prices dataset. Each sample in the dataset corresponds to an area in california. The attributes are latitude, longitude, median age of houses in the area (in years), total number of rooms in the area, total number of bed rooms in the area, population of the area, number of households in the areas, median income in the area (in tens of thousands of dollars), the area's proximity to ocean and the median house value.

###Get Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#HOUSING_PATH = "/content/drive/MyDrive/myLMSCourses/ML/2021_22_evenSemBatch/Practicals/1_exercise1/"
HOUSING_PATH = "/content/drive/MyDrive/Machine_Learning" #upload the housing.tgz file into the drive and give the path here


In [ ]:
import os
import tarfile
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
  tgz_path = os.path.join(housing_path, "housing.tgz")
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path=housing_path)
  housing_tgz.close()
  csv_path = os.path.join(housing_path, "housing.csv")
  return pd.read_csv(csv_path)

housing = load_housing_data()

###Peek into Data

In [ ]:
# look at the top five rows of data using dataframe's head method
# Your code below
housing.head()

In [ ]:
# Get a quick description of data using dataframe's info method; make a note of attributes with missing values in this notebook below this cell using a markdown cell.
# Your code below
housing.info()

Attributes with missing values:

total_bedrooms

In [ ]:
# What kind of attribute is ocean_proximity? Discrete or continuous? If discrete, use value_counts method on the column corresponding
# to ocean_proximity and get a description?
# Your code below
housing["ocean_proximity"].value_counts()


In [ ]:
housing["ocean_proximity"].describe()

ocean_proximity is a categorical attribute.

It is discrete.

In [ ]:
# Use describe method on the dataframe to get a summary of the numerical attributes
housing.describe()

In [ ]:
# Numerical attributes can also be described using histograms
# Observe how many attributes are thick tailed? What about the scales of the attributes? Are they uniform or vastly different?
%matplotlib inline
import matplotlib.pyplot as plt
#use hist method on dataframe
# Your code below
housing.hist(bins=100, figsize=(20,15))
plt.show()



### Thick-tailed

  A thick-tailed attribute is one whose distribution has a higher probability of extreme values than a normal distribution.

  Thick-tailed attributes: latitude, housing_median_age, median_house_value

 Scales of the attributes are vastly different


###Create Test Data
Why create test set now, right at the beginning? The reason is to avoid data snooping bias. That is, the more we look into data, our brain is powerful to capture the pattern seen in that data and will naturally influence our choice of the model. But that model may not generalize well during deployment since it was chosen simply based on some pattern seen in some sample data. So, it is better to separate out test set right in the beginning and keep it only for testing. We can do a pure random split of data into train and test set. For this you can look at train_test_split class in sklearn.model_selection. But, suppose your manager told that median income is an important attribute for predicting house price. Then, you would want to split data so that it reflects the various categories of median income in both train and test data. This is called as stratified sampling. Of course, median income is right now a numeric attribute. You have to create a new categorical attribute called income_cat, use that to split data into train and test sets, and then remove the income_cat attribute. To create income_cat, we can look at its respective histogram above and find that most median incomes are clustered around 1.5-6 (i.e \$15000-\$60000). So our categories could be 0-1.5, 1.5-3, 3-4.5, 4.5-6, >6.

In [ ]:
import numpy as np
# create income_cat attribute as described above
# housing["income_cat"] = # fill the code here; use cut method in pandas

categories = [0,1.5,3,4.5,6,float('inf')]
titles = ["very low", "low", "average", "high", "very high"]
housing["income_cat"] = pd.cut(housing.median_income, bins = categories, labels = titles, include_lowest = True)


In [ ]:
housing.head()

In [ ]:
# create train test stratified split (80-20 split) using income_cat attribute; use the train_test_split() method in sklearn.model_selection module
from sklearn.model_selection import train_test_split

# Your code below
X = housing
y = housing["median_house_value"]
income_cat = housing["income_cat"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=income_cat)


In [ ]:
# to check if the stratified split worked, compute and display the proportions of income categories in the test set and the whole dataset, and compare
# and make your observations in a markdown cell below.

# Your code below
# print(X_test["income_cat"].value_counts()/len(X_test))
# print(housing["income_cat"].value_counts()/len(housing))

# ----------------------------------------------------------

# Compute proportions of income categories in the whole dataset
income_cat_proportions = income_cat.value_counts(normalize = True)

# Compute proportions of income categories in the test set
test_set_proportions = X_test["income_cat"].value_counts(normalize = True)

# Print the proportions
print(income_cat_proportions)
print(test_set_proportions)


### Good Stratified split

The distribution of values of income_cat attribute in the whole data set and the test is preserved. Therefore this is a good stratified split.

In [ ]:
# drop the income_cat attribute from both train and test set; use dataframes's drop method
# Your code below

X_train = X_train.drop("income_cat", axis=1)
X_test = X_test.drop("income_cat", axis=1)





In [ ]:
print(X_train.shape)
print(X_test.shape)

### Explore Data
We will explore train data more to gain more insights. We will not touch test data. It will only be used at the end when we build a model and we are ready to test it. Even for exploring train set, to be on the safer side, we will make a copy of it. We will first visualize train data using scatter plot. See the plot below carefully. A lot of information has been embedded into it. Answer the questions given in comments.

In [ ]:
housing = X_train.copy()

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.8, s=housing["population"]/100,
         label="population", figsize=(10,7), c="median_house_value", cmap=plt.get_cmap("jet"),
         colorbar=True)

plt.legend()

# What does the size of the scatter point indicate? What does the color of the scatter point indicate? What is the relationship
# between ocean_proximity and median_house_value (note that the ocean is towards the bottom left in the plot)? Is there relationship between population and median_house_value?
# Describe the role of alpha parameter in the dataframe's plot method.

###Observations.


*   The bigger the size of scatter point, the more the
 population
is residing at that particular location of scatter point.
*   The color of scatter point indicates the median_house_value whose color code is given by the colorbar.
*   Most of the houses that are near the water bodies have high median_house_value.
*   There is no significant relationship between population and median_house_value.
*  Alpha is a parameter of transparency

   0-fully transparent

   1-fully opaque

















We will now look at linear correlations between median_house_value and all other attributes. Complete the code below  and answer the questions given in comments. In case you are not familiar with the concept of Perason's correlation, read about it.

In [ ]:
# use dataframe's corr method to get correlation matrix of every pair of attributes
# fill code here
corr_matrix = housing.corr(numeric_only=True)  # Add numeric_only=True to handle non-numeric columns
corr_matrix


# extract only median_house_value column from corr_matrix and sort it in descending order
# for sorting, use pandas series method sort_values
# Your code below
corr_matrix["median_house_value"].sort_values(ascending=False)

# Which attribute correlates positively highly with median_house_value? Are there attributes which have negligible linear correlations
# with median_house_value? What about negative linear correlations? Does a correlation value of zero or close to zero mean absolutely no relationship?

median_income correlates positively highly with median _house_value.

Yes, there are attributes that have negligible linear correlation with median_house_value. Some of them are the population of the block, number of households in the block, longitude, and total_bedrooms.

No it does not mean there is no relationship at all. There can be some non-linear relationship.



In [ ]:
 # Since median_income highly correlates with median_house_value, let's focus on that.
 # Display a scatter plot of median_income vs median_house_value
 # Your code below
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.5, s=housing["population"]/100,
        label="population", figsize=(10,7), cmap=plt.get_cmap("jet"),)

#  c="population", colorbar=True
# housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.8, s=housing["population"]/100,
#          label="population", figsize=(10,7), c="median_house_value", cmap=plt.get_cmap("jet"),
#          colorbar=True)
# Does the plot reveal anything? Change alpha value and see.

In [ ]:
#The below code is just for understanding the data

housing[(housing["median_income"] >= 1) & (housing["median_income"] <= 6) & (housing["median_house_value"] > 480000)]["ocean_proximity"].value_counts()

###Observations.

median_income - Median income for households within a block (measured in ten thousand US Dollars.)


medianHouseValue -	Median house value for households within a block (measured in US Dollars)


We can make out a linear relation between median_income and median_house_value. The reason why even less median_income population has more median_house_value is because of the ocean_proximity factor that we didn't include in our correlation matrix.

In [ ]:
# Attribute like total_rooms, total_bedrooms, population are too general to relate to house price
# Above correlations also show this
# Why not create population per household, rooms per household, ratio of bed_rooms to rooms?
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"] # Fill your code here
housing["population_per_household"] = housing["population"]/housing["households"] # Fill your code here


In [ ]:
# Now see the correlations of median_house_value to all the attributes including new attributes introduced above
# Your code below
corr_matrix = housing.corr(numeric_only=True)
corr_matrix["median_house_value"].sort_values(ascending=False)

# Any new observations ??


### Observations.
rooms_per_household has a better positive correlation value than total_rooms with the median_house_value.

### Prepare (Preprocess) Data
We have explored train data and found that some derived attributes may be useful. Now, we will need to preprocess and prepare the data before building the model to process the prepared data. For the data under consideration we need to do the following:
 1. Separate the label (median_house_value) and the rest of the attributes
 2. Fill the missing values in total_bedrooms attribute with median of rest of of the entries in it.
 3. As we noted earlier, scales of attributes are vastly different. Bring all of them to uniform scale using standardization. That is, for each attribute (not the label), subtract mean of it from each of its entry and divide by its standard deviation. This way, all attributes will become zero centred and will have its scale in standard deviation units. There is another way called as normalization to bring all attributes into uniform scale. Here, for each attribute, subtract min of that attribute from each of its entry and divide by max minus min of that attribute. This will ensure the range of attribute is in [0, 1]. Standardization is preferred over normalization generally since normalization are more sensitive to outliers than standardization. For eg, guess what will happen if one attribute had all the values in the range 0-15 except for one (which is 100). In normalization, the range will simply get crushed to [0, 0.15] whereas in standardization it is more likely to be wider.
 4. Note that ocean_proximity is a categorical attribute. We need to convert it to numerical attribute before building the model. One way of doing this is to simply assign 0, 1, 2... to the categories. This might work in certain situations but not always. For eg, in the situation here, assigning 0 to <1H OCEAN, 1 to INLAND, 2 to NEAR OCEAN, 3 to NEAR BAY and 4 to ISLAND would mean to the model that NEAR OCEAN and INLAND are closer than NEAR OCEAN and <1H OCEAN, which is wrong. Instead, since there are 5 categories, we will represent each category by a binary vector of length 5 such that one unique component of this binary vector is 1 and rest are zero for this category, and so on. In other words, <1H OCEAN will be represented by 10000, INLAND will be represented by 01000, and so on. This is called as one hot encoding. We will be converting ocean_proximity into one hot encoding representation. One hot encoding will not be efficient if the number of catgories are very large. For eg, imagine that the categorical attribute is vocabulary and the number of categories are number of words in the vocabulary. But in the case here, we have only 5 categories. So, no problem with one hot encoding.

In [ ]:
# First separate the label median_house_value
# Fill your code here; make a copy of the median_house_value
# drop the median_house_value from housing dataframe inplace
# Your code below
housing_labels = housing["median_house_value"].copy()
housing.drop("median_house_value", axis=1, inplace = True)

# Let me save my housing_labels as csv file in my G_drive at mentioned path.
housing_labels_df = pd.DataFrame(housing_labels)

# Specify file path
file_path = '/content/drive/My Drive/Machine_Learning/housing_labels_train.csv'  # Replace with your desired path

# Save the file
housing_labels_df.to_csv(file_path, index=False, header=['median_house_value'])  # Set header to column name


In [ ]:
# Fill any missing value with median value of the attribute it corresponds to
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = "median")  # Fill your code here; instantiate SimpleImputer object with median strategy

# Note that the imputer will be automatically used on numerical attributes once we set up the pipeline.
# Right now, we have instantiated it.

In [ ]:
# Standardization
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()# Fill your code here; instantiate StandardScaler object


In [ ]:
# set up the pipeline consisting of above two transforms which deal with numerical attributes
from sklearn.pipeline import Pipeline

#Fill your code here; instantiate Pipeline object with the above two transforms in order
numerical_pipeline = Pipeline([
    ('imputer', imputer),
    ('std_scaler', std_scaler)
])


# The way this pipeline works is as follows. When we call its fit_transform method on the data devoid of categorical attributes,
# it will first call the fit_transform method of the imputer object. The fit_transform method of the imputer object will first call
# fit method of the imputer object which will compute the median of all numerical attributes, respectively (based on their non-null values)
# and store it in statistics_ public variable and return itself. Then the transform method from SimpleImputer class is called on the returned
# imputer object which will fill the missing values with the respective median value picked up from statistics_ variable.
# The transform method returns the transformed data. This will then go as input to the fit method of the std_scaler transform in the
# pipeline. The fit method will compute mean and std dev with respect to each attribute and store them in mean_ and scale_ public variables,
# and return the std_scaler object itself. Then the transform method from the StandardScaler class is called on the returned std_scaler
# object which will do the standardization on each numerical attribute, respectively. The transformed data will be returned by the pipeline.

# Currently we are not calling the fit_tranform method on the pipeline because that will require data to be devoid of categorical
# attributes. As already explained earlier, we will not do that. Instead we will set up another pipeline which will handle
# this pipeline and the transforms on categorical attributes automatically.



In [ ]:
# represent ocen_proximity attribute in one hot vector encoding
from sklearn.preprocessing import OneHotEncoder

categorical_encoder = OneHotEncoder()# Fill your code here; instantiate OneHotEncoder object


In [ ]:
# set up the ColumnTransformer pipeline that will automatically deal with both numerical pipeline and OneHotEncoder transform.
from sklearn.compose import ColumnTransformer
        # Fill your code here; get the list of numerical attribute names - first get all attribute names; then remove
                     # ocean proximity; Note that remove method of list does inplace removal and returns none if success; if you are
                     # using remove method, don't chain it with list creation; do it in the next line separately.
mylist = list(housing.columns)
mylist.remove("ocean_proximity")
numerical_attribs =  mylist
print(mylist)

# Fill your code here; create a list with one entry which is ocean_proximity

categorical_attribs = ["ocean_proximity"]

# full_pipeline = # Fill your code here; instantiate ColumnTransformer pipleline with numerical pipeline followed by one hot encoder
full_pipeline =  ColumnTransformer([("numerical", numerical_pipeline, numerical_attribs),
                                      ("categorical", categorical_encoder, categorical_attribs)
  ])


# housing_prepared = # Fill your code here; call the fit_transform method on the ColumnTransformer pipeline object
housing_prepared = full_pipeline.fit_transform(housing)

# housing_prepared is normalized X_train without "median_house_value" and more columns have been added

# Get the feature names after transformation
feature_names = numerical_attribs.copy()
feature_names.extend(full_pipeline.named_transformers_['categorical'].get_feature_names_out())

# Save to CSV with updated header
housing_prepared_df.to_csv(file_path, index=False, header=feature_names)

# Let me save housing_prepared as csv file in my Google Drive at specified path
housing_prepared_df = pd.DataFrame(housing_prepared)

# Specify the path in your Google Drive
file_path = '/content/drive/MyDrive/Machine_Learning/housing_prepared_train.csv'

# Save to CSV
housing_prepared_df.to_csv(file_path, index=False, header = feature_names)

# The pipeline works similar to what was already explained earlier.
housing_prepared.shape


### Select and Train Model
Now that the data is prepared, we need to select a model, train it and see how it performs. Sometimes application and data can guide towards model selection. For example, if the volume of data is very large and the application is computer vision or natural language processing, neural networks are very powerful models. As one gains more experience, it becomes relatively easy to identify the family of models that would suit a particular problem at hand. However, one may not be able to nail down to the best model right at the first instance, even with experience. It is always an iterative process. Multiple models have to compared before finalizing on the model to be deployed. For now, we will only train a linear regression model in this notes.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
# Fill your code here; instantiate LinearRegression object


# Now fit the data using fit the method on linear regression object
# Your code below
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# let's try out this model on few instances from train data

sample_data = housing.head()# Fill your code here; get first five rows from housing dataframe
# sample_labels = # Fill your code here; get first five labels from housing_labels pandas series
sample_labels =  housing_labels.head()
# sample_data_prepared = # Fill your code here; transform the sample_data using full_pipeline.
                        # Note that you need to only transform this data; not fit. Fitting was already done.
sample_data_prepared = full_pipeline.transform(sample_data)

# predictions = # Fill your code here; get the predictions on sample_data using predict method on the already fitted linear regression object
predictions = lin_reg.predict(sample_data_prepared)
print("Predictions: ", predictions)
print("Groundtruth: ", list(sample_labels))


We can see that the predictions are not very close. Infact they are off by around 27% on average. We can check the performance on the entire training set. Before doing that, we need a way of quantifying the performance. There are three standard performance measure for linear regression viz. mean absolute error (MAE), mean squared error (MSE) and root mean squared error (RMSE).

MAE = $\frac{1}{n}\sum_{i=1}^{n} |(y^i-\hat y^i)|$

MSE = $\frac{1}{n}\sum_{i=1}^{n} (y^i-\hat y^i)^2$

RMSE = $\sqrt{\frac{1}{n}\sum_{i=1}^{n} (y^i-\hat y^i)^2}$

The linear regression model can be fit by minimizing any of the above errors. Note that MSE punishes large errors more severely than MAE. So, MSE is sensitive to outliers than MAE. Further, minimizing MSE results in units of the response getting squared and so intrepretation becomes difficult. Hence, RMSE is preferred over MSE which maintains MSE. We will use RMSE below to see the performance, but we can try with any of the above.

In [ ]:
housing_labels.describe()

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, predictions)
lin_rmse = np.sqrt(lin_mse)
print(f"RMSE: {lin_rmse:.2f}")

What this means is that the predictions are off by \$67,629 on the training set. Is this a good performance on the train set? Compare this offset with the range of median_house_value between  $25^{th}$ and $75^{th}$ percentile we had obtained earlier using dataframe's describe method. In fact the model is underfitting the data. We need to look for more complex models, like decision tree or random forest.


### Observation

The range of median_house_value between  25th  and  75th  percentile we had obtained earlier using dataframe's describe method is 145725

The above model's RMSE is substantial (almost 45%) when compared to middle 50% the house prices (varying from around 1 lakh to around 2.5 lakhs). So it did not perform very well on training set.



### Fine-tune Model
We saw that the model was underfitting. In some cases, the model may overfit (i.e give 0 train error). We know that we should look for better models. But, since we have now studied only regression, we need to wait until we pick other models. In any case, in the end-end ML solution, once we fit the model, we try to finetune its hyperparameters using a validation dataset. That is, we try different values of hyperparameters for the model, check its performance on the validation set and choose the one which gives the best performance. The validation dataset should neither intersect with train nor the test set. If it intersects with trainset, the model is likely to overfit depending on the amount of intersection. And we know that the test set is not to be touched at all during training. In linear regression, the only hyper parameter we can think of tuning is whether to have the intercept parmeter or not. However, in the example we are studying here, the performance is not going to change (check for yourself). So, we will not not be doing any hyperparameter finetuning here. But remember that this step is important in the end-end ML solution. We will understand more about hyperparameters as we go along.

Of course, we can also a lot of analysis like what we had one in our earlier linear regression notes to finetune the model. I leave that as optional exercise.

### Evaluation on Test Set
Assume that the linear regression model is the best model we have got after all the above steps. Now, we will test the model on the test set.

In [ ]:
X_test_drop = X_test.drop("median_house_value", axis = 1)# Fill your code here; drop median_house_value from strat_test_set
test_labels = X_test["median_house_value"].copy() # Fill your code here; get a copy of median_house_value column from strat_test_set


housing_labels_test_df = pd.DataFrame(test_labels)

# Specify file path
file_path = '/content/drive/My Drive/Machine_Learning/housing_labels_test.csv'  # Replace with your desired path

# Save the file
housing_labels_test_df.to_csv(file_path, index=False, header=['median_house_value'])  # Set header to column name

# add the derived attributes rooms_per_household, bedrooms_per_room, population_per_household
# Your code below
X_test_drop["rooms_per_household"] = X_test_drop["total_rooms"] / X_test_drop["households"]
X_test_drop["bedrooms_per_room"] = X_test_drop["total_bedrooms"] / X_test_drop["total_rooms"]
X_test_drop["population_per_household"] = X_test_drop["population"] / X_test_drop["households"]

X_test_prepared = full_pipeline.transform(X_test_drop)# Fill your code here; transform X_test through full pipeline

# Let me save housing_prepared as csv file in my Google Drive at specified path
housing_prepared_test_df = pd.DataFrame(X_test_prepared)

# Specify the path in your Google Drive
file_path = '/content/drive/MyDrive/Machine_Learning/housing_prepared_test.csv'

# Save to CSV
housing_prepared_test_df.to_csv(file_path, index=False, header = feature_names)

test_predictions = lin_reg.predict(X_test_prepared)# Fill your code here; get test predictions
test_mse = mean_squared_error(test_labels, test_predictions)# Fill your code here; compute test mse
test_rmse = np.sqrt(test_mse)# Fill your code here; compute test rmse

print(f"RMSE: {test_rmse:.2f}")

### Pre-launch
Before you launch the model, you need to present it to your team to get their final approval. So, as a pre-launch exercise, you will submit the completed assignment to me.

###Launch
You can launch your model as webservice through a REST API or through some other means. We will not dicuss that now.